In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting

In [2]:
order_products=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/order_products.csv')
orders=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/orders.csv')

## Group customer by their max order number

In [3]:
df=orders.groupby('user_id')['order_number'].aggregate('max').reset_index()

In [4]:
df['eval_set']=0

In [5]:
df.head(10)

,user_id,order_number,eval_set
0,1,11,0
1,2,15,0
2,3,13,0
3,4,6,0
4,5,5,0
5,6,4,0
6,7,21,0
7,8,4,0
8,9,4,0
9,10,6,0


## Joining grouped table to orders table by user_id,order_number

In [127]:
df1=pd.merge(orders,df, how='left', left_on=['user_id','order_number'], right_on=['user_id','order_number'])

In [128]:
df1

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,NaN
1,2398795,1,2,3,7,15.0,NaN
2,473747,1,3,3,12,21.0,NaN
3,2254736,1,4,4,7,29.0,NaN
4,431534,1,5,4,15,28.0,NaN
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,NaN
3421079,1854736,206209,11,4,10,30.0,NaN
3421080,626363,206209,12,1,12,18.0,NaN
3421081,2977660,206209,13,1,12,7.0,NaN


## Fill in 'Prior' for the null value as they are not the lastest order

In [129]:
df1.eval_set.fillna('Prior',inplace=True)

In [130]:
Non_prior = df1[df1["eval_set"] ==0]
Non_prior.drop('eval_set',axis=1,inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## Join distinct order_products on to non prior data

In [106]:
distinct_order_id=order_products[['order_id']].drop_duplicates()
distinct_order_id['eval_set']=0

In [115]:
distinct_order=pd.merge(Non_prior,distinct_order_id,how='left',on='order_id')
distinct_order

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,1187899,1,11,4,8,14.0,0.0
1,1492625,2,15,1,11,30.0,0.0
2,2774568,3,13,5,15,11.0,NaN
3,329954,4,6,3,12,30.0,NaN
4,2196797,5,5,0,11,6.0,0.0
...,...,...,...,...,...,...,...
206204,1716008,206205,4,1,16,10.0,0.0
206205,1043943,206206,68,0,20,0.0,NaN
206206,2821651,206207,17,2,13,14.0,NaN
206207,803273,206208,50,5,11,4.0,NaN


In [116]:
distinct_order.eval_set.fillna('Train',inplace=True)

In [117]:
distinct_order.replace(0, 'Test', inplace=True)
distinct_order

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,1187899,1,11,4,8,14.0,Test
1,1492625,2,15,1,11,30.0,Test
2,2774568,3,13,5,15,11.0,Train
3,329954,4,6,3,12,30.0,Train
4,2196797,5,5,Test,11,6.0,Test
...,...,...,...,...,...,...,...
206204,1716008,206205,4,1,16,10.0,Test
206205,1043943,206206,68,Test,20,Test,Train
206206,2821651,206207,17,2,13,14.0,Train
206207,803273,206208,50,5,11,4.0,Train


## Join all the tables

In [131]:
prior = df1[df1["eval_set"] == "Prior"]

In [132]:
order_new=pd.concat([prior,distinct_order])

In [133]:
order_new

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
206204,1716008,206205,4,1,16,10.0,Test
206205,1043943,206206,68,Test,20,Test,Train
206206,2821651,206207,17,2,13,14.0,Train
206207,803273,206208,50,5,11,4.0,Train


In [81]:
orr['eval_set'].value_counts()

Prior    32434489
Train     1384617
Test        75000
Name: eval_set, dtype: int64